## pytopomat Example Notebook
This notebook will show you how to
* Obtain symmetry operation eigenvalues with vasp2trace and irvsp.
* Compute topological invariants directly from output files.
* Submit workflows for high-throughput band topology calculations.

In [40]:
import os

from pytopomat.analyzer import Vasp2TraceOutput, BandParity, IRVSPOutput, IRVSPCaller

In [27]:
test_dir = "test_files/"

In [34]:
Bi = Vasp2TraceOutput(test_dir + "Bi_trace_soc.txt")
# bp = BandParity({"up": Bi})

### irvsp to compute irreducible representations
Install irvsp from: https://github.com/zjwang11/irvsp/blob/master/src_irvsp_v2.tar.gz  
following the instructions here: https://arxiv.org/pdf/2002.04032.pdf  

Run a calculation with ISYM = 1 or 2 (to print space group operations) and LWAVE = .TRUE.  
Make sure the crystal is given in a standard setting, e.g. by doing `phonopy --tolerance 0.01 --symmetry -c POSCAR`

In [41]:
# In the directory with OUTCAR, WAVECAR, and POSCAR
ic = IRVSPCaller('.')
output = ic.output

RuntimeError: IRVSPCaller requires irvsp to be in the path.
Please follow the instructions in https://arxiv.org/pdf/2002.04032.pdf
https://github.com/zjwang11/irvsp/blob/master/src_irvsp_v2.tar.gz

In [36]:
irvsp_out = IRVSPOutput(test_dir + "CrO2_outir.txt")

In [39]:
# Inspect the parity eigenvalues for the first 5 valence bands at the Gamma point
irvsp_out.parity_eigenvals["gamma"]["inversion_eigenval"][:5]

[1.0, 1.0, -2.0, 1.0, 1.0]

### Workflows for high-throughput band topology
Generating wflows for topology calculations is easy with pytopomat.

In [42]:
from pymatgen import Structure, Lattice

In [43]:
bcc_bi = Structure.from_spacegroup("Im-3m", Lattice.cubic(3.453), ["Bi"], [[0, 0, 0]])

Vasp2Trace wflows do band structure calculations at TRIM kpoints and compute the parity eigenvalues for obtaining topological invariants. They also generate "trace.txt" for non-spin-polarized calculations and "trace_up.txt" and "trace_dn.txt" files for spin-polarized calculations, which can be used to check the topological character of a material with Topological Quantum Chemistry.
https://www.cryst.ehu.es/cgi-bin/cryst/programs/topological.pl

In [46]:
from pytopomat.workflows.core import wf_vasp2trace_nonmagnetic, wf_vasp2trace_magnetic

wf1 = wf_vasp2trace_nonmagnetic(bcc_bi)

FileNotFoundError: [Errno 2] No such file or directory: '/home/nfrey/pmg_env/lib/python3.6/site-packages/atomate/vasp/workflows/base/library/vasp2trace_nonmagnetic.yaml'

In [52]:
bcc_bi.add_site_property("magmom", [5.0, 5.0])
wf2 = wf_vasp2trace_magnetic(bcc_bi)

NameError: name 'os' is not defined

The Z2Pack wflow is a more general wflow that uses hybrid Wannier charge centers to compute Z2 indices and Chern numbers on inequivalent TRI planes in the 3D Brillouin zone.

In [53]:
from pytopomat.workflows.core import Z2PackWF

wf = Z2PackWF(bcc_bi, symmetry_reduction=False)